#Digital Marketing Analytics Assignment 2 - Segmentation and Targeting

Jasmine Gohil & Khushi Jasrapuria

In [ ]:
!pip install stargazer

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer

In [ ]:
df = pd.read_csv("/content/PS2-2024.csv")
df

,id,trans_after,revenue_after,test_coupon,non_male,income,channel_acq,num_past_purch,spent_last_purchase
0,AB_1,0,0.00,0,0,high,2,6,62.99
1,AB_2,0,0.00,1,0,high,1,2,53.99
2,AB_3,0,0.00,1,0,high,2,3,88.98
3,AB_4,0,0.00,0,1,low,2,1,68.99
4,AB_5,0,0.00,1,1,high,3,3,66.49
...,...,...,...,...,...,...,...,...,...
4995,AB_4996,1,51.49,0,1,high,2,3,51.49
4996,AB_4997,0,0.00,1,0,high,1,0,0.00
4997,AB_4998,0,0.00,1,1,high,1,1,60.49
4998,AB_4999,0,0.00,1,0,low,1,0,0.00


In [ ]:
df['channel_acq'] = df['channel_acq'].astype("category").cat.rename_categories({
    1: "Google", 2: "Facebook", 3: "instagram", 4:"Referal", 5:"Other"
    })

In [ ]:
df['channel_acq'].value_counts()

channel_acq
Google       2035
instagram    1552
Facebook     1059
Referal       238
Other         116
Name: count, dtype: int64

# Question 1

Your colleague, Randy, proposed targeting the following segment: consumers who
were acquired from the Instagram channel and have made purchases on the company
website twice or fewer prior to the experiment (hereafter, Segment A). As the first step,
you want to see if the experiment passes the randomization check, in particular
regarding the referral channel composition (i.e. which channel they came from) and
the pre-experiment purchase counts among the treatment and control users.

##Question 1(a)

Use a single regression (instead of multiple separate regressions) to assess
whether the experiment and control customers are statistically similar in terms of
their acquisition channel composition and pre-experiment purchase counts. Note
that you need to compare the composition across all acquisition channels, not just
the percentage of customers acquired from Instagram. Include the point estimates
and confidence intervals of the treatment-control differences regarding channel
composition and pre-experiment purchase counts. (5 points)

In [ ]:
X = df[['test_coupon', 'channel_acq', 'num_past_purch']]
X = sm.add_constant(X)

In [ ]:
y = df['revenue_after']

In [ ]:
ran_check1 = smf.ols(formula='test_coupon ~ channel_acq + num_past_purch', data=df).fit()
result_ran_check1 = Stargazer([ran_check1])
result_ran_check1

In [ ]:
 #Point Estimates
 ran_check1.params

Intercept                   0.489974
channel_acq[T.Facebook]     0.018119
channel_acq[T.instagram]   -0.001188
channel_acq[T.Referal]      0.029159
channel_acq[T.Other]        0.004032
num_past_purch              0.002664
dtype: float64

In [ ]:
#Confidence Intervals
ran_check1.conf_int()

,0,1
Intercept,0.465658,0.514289
channel_acq[T.Facebook],-0.019038,0.055275
channel_acq[T.instagram],-0.034233,0.031857
channel_acq[T.Referal],-0.038030,0.096348
channel_acq[T.Other],-0.089576,0.097641
num_past_purch,-0.002755,0.008082


##Question 1(b)
Does the experiment pass the randomization check based on your calculation above? (5 point)

The confidence intervals and coefficients for all values of the parameters have zero in them. Moreover, even the p values are more than 0.05, meaning results are statistically significant. This means that our experiment does pass the randomised check.

#Question 2
Your team now wants to evaluate the effectiveness of this targeting strategy using the experiment data you have. The regression result looks like the following

In [ ]:
df['segment_A'] = ((df['num_past_purch'] <= 2) & (df['channel_acq'] == 'instagram')).astype('float')
hetero_effect_cust = smf.ols(formula='revenue_after ~ test_coupon * segment_A', data=df).fit()
Stargazer([hetero_effect_cust])

## Question 2(a)
What is the meaning of the Intercept coefficient “8.541”? Which customer group does this number correspond to? (5 points)

## Answer 2(a)
The Intercept coefficient is 8.541.
This means that when both test_coupon and segment_A are zero, the expected revenue_after is approximately 8.541 units.

* The customer group corresponding to this Intercept coefficient includes those who have not received the coupon (test_coupon = 0) and do not belong to Segment A (segment_A = 0).
* Therefore, this Intercept coefficient represents the baseline revenue level for customers who have not received the coupon and were not acquired through Instagram with few past purchases.

##Question 2(b)
What is the incremental revenue per user when you send the coupon to the customers outside segment A? (5 points)

## Answer 2(b)

Given the regression results:

* The coefficient for the test_coupon variable is -0.652.
This coefficient represents the change in revenue_after associated with sending the coupon to customers outside segment A, while holding all other variables constant.

* Therefore, the incremental revenue per user when sending the coupon to customers outside segment A is approximately -0.652 units.

* The negative coefficient indicates a decrease in revenue_after when the coupon is sent to customers outside segment A.

##Question 2(C)
What is the incremental revenue per user when you send the coupon to segment A? (5 points)

## Answer 2(c)

Given the regression results:

* The coefficient for the interaction term test_coupon:segment_A is 2.756.
This coefficient represents the change in revenue_after associated with sending the coupon to segment A customers, while holding all other variables constant.

* Therefore, the incremental revenue per user when sending the coupon to segment A is approximately 2.756 units.

* This positive coefficient indicates an increase in revenue_after when the coupon is sent to segment A customers.

# Question 3
Another colleague in your team, Greg, ran a different regression in order to understand how customers coming from different channels with different prior purchase counts status respond to the coupon differently. He sent the following regression results to the team:

In [ ]:
df['instagram'] = (df['channel_acq'] == 'instagram').astype('float')
df['few_prior_purch'] = (df['num_past_purch'] <= 2).astype('float')
hetero_effects = smf.ols(formula='revenue_after ~ test_coupon * instagram * few_prior_purch', data=df).fit()
Stargazer([hetero_effects])

## Question 3(a)
What is your interpretation on the coefficient `test_coupon: Instagram`? (5 points)

## Answer 3(a)

Positive Coefficient: The positive coefficient (3.881) indicates that when a customer is exposed to both the test coupon and Instagram, there might be a positive effect on their revenue_after compared to customers who are not exposed to either. In other words, the combined effect of the test coupon and Instagram might lead to higher revenue.

##Question 3(b)

Based on this regression table, what is the incremental revenue per user when you
target coupons to customers acquired through Instagram and have made two prior
purchases or fewer? Explain how you use the numbers in this table to calculate the
results. (5 points)

## Answer 3(b)

Based on the regression table, the incremental revenue per user when you target coupons to customers acquired through Instagram and have made two prior purchases or fewer is  -2.387.

This term captures the specific effect of targeting coupons to customers acquired through Instagram who have made two prior purchases or fewer. In the table, this coefficient is  -2.387.

Since the coefficient is negative, it indicates that there is a negative incremental revenue associated with targeting coupons to this specific customer segment. In other words, on average, these customers generate  $2.387 less in revenue after receiving a coupon compared to customers who were not targeted with a coupon.

## Question 3(c)

Compare the results here with your conclusion in 2(c). Do the results reported by
Randy and Greg contradict each other? If they do contradict each other, which
result do you trust? Explain your reasoning. (5 points).

# Answer 3(c)
The results compared to that of 2(c), and reported by Randy and Greg, contradict each other. Clearly, Randy's result give a better idea as they are directly capturing the right intended segment.It makes sense as having coupons results in lesser immediate revenue compared to revenue without coupons. On the other hand, Greg's results give another point of view saying there are other factors contributing to the result regarding channels and also fewer prior purchases, but unlike that of Randy's results, Greg's are not specific. Hence, its better to trust Randy's results.

# Question 4  

Evaluate the effectiveness of targeting segment A:

## Question 4(a)

Calculate the total incremental conversions when you send the coupon to all
treatment customers in Segment A. Report point estimate, standard error, and the
95% confidence interval. (8 points)

In [ ]:
df['segment_A'] = ((df['num_past_purch'] <= 2) & (df['channel_acq'] =='instagram')).astype('float')

In [ ]:
df['segment_A'].value_counts()

segment_A
0.0    3929
1.0    1071
Name: count, dtype: int64

In [ ]:
segment_A_data = df[df['segment_A'] == 1]

mean_trans_with_coupon_A = segment_A_data[segment_A_data['test_coupon'] == 1]['trans_after'].mean()

# Calculate mean transaction after where test_coupon is 0 for Segment A customers
mean_trans_without_coupon_A  = segment_A_data[segment_A_data['test_coupon'] == 0]['trans_after'].mean()

n_with_coupon_A = len(segment_A_data[segment_A_data['test_coupon'] == 1])
n_without_coupon_A = len(segment_A_data[segment_A_data['test_coupon'] == 0])

# Calculate transaction after difference
trans_difference_A = (mean_trans_with_coupon_A- mean_trans_without_coupon_A) * n_with_coupon_A

std_dev_with_coupon_A = segment_A_data[segment_A_data['test_coupon'] == 1]['trans_after'].std()
std_dev_without_coupon_A = segment_A_data[segment_A_data['test_coupon'] == 0]['trans_after'].std()

se = np.sqrt((std_dev_with_coupon_A ** 2 / n_with_coupon_A) + (std_dev_without_coupon_A ** 2 / n_without_coupon_A)) * n_with_coupon_A

CI_lower_A = trans_difference_A - 1.96 * se
CI_upper_A = trans_difference_A + 1.96 * se


print('Point Estimate of the Mean Difference: ', np.round(trans_difference_A, 3))
print('Standard error: ', np.round(se, 3))
print('95% CI: [', np.round(CI_lower_A, 3), ', ', np.round(CI_upper_A, 3), ']')

Point Estimate of the Mean Difference:  28.915
Standard error:  11.944
95% CI: [ 5.506 ,  52.324 ]


##Question 4(b)

 Calculate the total incremental revenue when you send the coupon to all treatment customers in Segment A. Report point estimate, standard error, and the 95% confidence interval. (8 points)

In [ ]:
# Filter data for customers in Segment A
segment_A_data = df[df['segment_A'] == 1]

# Calculate mean revenue after where test_coupon is 1 for Segment A customers
mean_rev_with_coupon_A = segment_A_data[segment_A_data['test_coupon'] == 1]['revenue_after'].mean()

# Calculate mean revenue after where test_coupon is 0 for Segment A customers
mean_rev_without_coupon_A  = segment_A_data[segment_A_data['test_coupon'] == 0]['revenue_after'].mean()

n_with_coupon_A = len(segment_A_data[segment_A_data['test_coupon'] == 1])
n_without_coupon_A = len(segment_A_data[segment_A_data['test_coupon'] == 0])

# Calculate rev after difference
rev_difference_A = (mean_rev_with_coupon_A- mean_rev_without_coupon_A) * n_with_coupon_A

std_dev_with_coupon_A = segment_A_data[segment_A_data['test_coupon'] == 1]['revenue_after'].std()
std_dev_without_coupon_A = segment_A_data[segment_A_data['test_coupon'] == 0]['revenue_after'].std()

se = np.sqrt((std_dev_with_coupon_A ** 2 / n_with_coupon_A) + (std_dev_without_coupon_A ** 2 / n_without_coupon_A)) * n_with_coupon_A

#Confidence Interval
CI_lower_A = rev_difference_A - 1.96 * se
CI_upper_A = rev_difference_A + 1.96 * se


print('Point Estimate of the Mean Difference: ', np.round(rev_difference_A, 3))
print('Standard error: ', np.round(se, 3))
print('95% CI: [', np.round(CI_lower_A, 3), ', ', np.round(CI_upper_A, 3), ']')

Point Estimate of the Mean Difference:  1114.762
Standard error:  659.158
95% CI: [ -177.188 ,  2406.712 ]


##Question 4(c)

Based on your calculation above, what is your assessment on the effectiveness of targeting segment A with coupons? (4 points)


## Answer 4(c)

**Transaction After:**

* The point estimate of the mean difference for transaction after is 28.915, indicating that sending coupons to customers in Segment A leads to an average increase of 28.915 units in transaction after.
* The standard error is relatively small compared to the point estimate, suggesting that the estimate is quite precise.
* The 95% confidence interval ranges from 5.506 to 52.324. Since this interval does not include zero, it indicates that the effect of sending coupons to Segment A customers on transaction after is statistically significant.

**Revenue After:**

* The point estimate of the mean difference for revenue after is 1114.762, indicating that sending coupons to customers in Segment A leads to an average increase of 1114.762 units in revenue after.
* The standard error is relatively large compared to the point estimate, suggesting some uncertainty in the estimate.
* The 95% confidence interval ranges from -177.188 to 2406.712. Since this interval includes zero, it suggests that the effect of sending coupons to Segment A customers on revenue after is not statistically significant.

**Conclusion:**

While sending coupons to Segment A customers appears to have a statistically significant positive effect on transaction after, the effect on revenue after is not statistically significant. Therefore, the effectiveness of targeting Segment A with coupons seems to vary depending on the outcome measure considered.

# Question 5

Another team member, Audrey, proposed to target a slightly different segment. She
reasoned that the company’s core customers are usually people in their 30s-40s, who
are more likely to use Facebook than Instagram. She then proposes to target customers
acquired from Facebook and have made purchases on the company website twice or
fewer prior to the experiment (hereafter Segment B).


##Question 5(a)

Calculate the user-level incremental revenue when you send the coupon to
customers in Segment B. Report point estimate, standard error, and the 95%
confidence interval. (8 points)

In [ ]:
df['segment_B'] = ((df['num_past_purch'] <= 2) & (df['channel_acq'] == 'Facebook')).astype('float')

In [ ]:
df['segment_B'].value_counts()

segment_B
0.0    4272
1.0     728
Name: count, dtype: int64

In [ ]:
# Filter data for customers in Segment B
segment_B_data = df[df['segment_B'] == 1]

# Calculate mean revenue where test_coupon is 1 for Segment B customers
mean_rev_with_coupon_B = segment_B_data[segment_B_data['test_coupon'] == 1]['revenue_after'].mean()

# Calculate mean revenue where test_coupon is 0 for Segment B customers
mean_rev_without_coupon_B = segment_B_data[segment_B_data['test_coupon'] == 0]['revenue_after'].mean()

# Calculate revenue difference
rev_difference_B = mean_rev_with_coupon_B - mean_rev_without_coupon_B

# Standard Error
n_with_coupon_B = len(segment_B_data[segment_B_data['test_coupon'] == 1])
n_without_coupon_B = len(segment_B_data[segment_B_data['test_coupon'] == 0])
std_dev_with_coupon_B = segment_B_data[segment_B_data['test_coupon'] == 1]['revenue_after'].std()
std_dev_without_coupon_B = segment_B_data[segment_B_data['test_coupon'] == 0]['revenue_after'].std()

se = np.sqrt((std_dev_with_coupon_B ** 2 / n_with_coupon_B) + (std_dev_without_coupon_B ** 2 / n_without_coupon_B))

# Confidence Interval
CI_lower_B = rev_difference_B - 1.96 * se
CI_upper_B = rev_difference_B + 1.96 * se

print('Point Estimate of the Mean Difference: ', np.round(rev_difference_B, 3))
print('Standard error: ', np.round(se, 3))
print('95% CI: [', np.round(CI_lower_B, 3), ', ', np.round(CI_upper_B, 3), ']')

Point Estimate of the Mean Difference:  2.642
Standard error:  1.313
95% CI: [ 0.069 ,  5.216 ]


## Question 5(b)

Calculate the total incremental revenue when you send the coupon to all treatment
customers in Segment B. Report point estimate, standard error, and the 95%
confidence interval. (8 points)

In [ ]:
n_B_segment = len(segment_B_data[segment_B_data['test_coupon'] == 1])

total_incremental_revenue_B = rev_difference_B * n_B_segment
print('Total Incremental Revenue Estimate: ', np.round(total_incremental_revenue_B, 3))

Total Incremental Revenue Estimate:  961.756


In [ ]:
se = (np.sqrt(np.std(segment_B_data[segment_B_data['test_coupon'] == 1]['revenue_after'])**2/n_with_coupon_B +
              np.std(segment_B_data[segment_B_data['test_coupon'] == 0]['revenue_after'])**2/n_with_coupon_B))

tCI_lower = np.round(rev_difference_B  - 1.96*se, 3)
tCI_upper = np.round(rev_difference_B  + 1.96*se, 3)

print('Standard error: ', np.round(se, 3))
print('95% CI: [', tCI_lower, ', ', tCI_upper, ']')

Standard error:  1.311
95% CI: [ 0.072 ,  5.212 ]


## Question 5(c)

Compare these results with your assessment of Randy’s proposal. What do you
conclude about the relative effectiveness of targeting Segment A vs. Segment B? (4
points)

## Answer 5(c)

**Segment Effect:**

The coefficients for segment_A and segment_B in the regression results represent the difference in revenue between customers in Segment A and Segment B, respectively, compared to customers not in those segments.
Both coefficients are negative, indicating that customers in both Segment A and Segment B tend to have lower revenue compared to customers not in those segments.

**Treatment Effect (Coupon Effect):**

The coefficients for test_coupon in both segments represent the average treatment effect of sending the coupon to all customers, regardless of segment. In the provided results, both coefficients are negative, suggesting that, on average, customers who receive the coupon have a decrease in revenue compared to those who do not receive the coupon.
Interaction Term (Test_coupon:Segment_B):

The coefficient for the interaction term test_coupon:segment_B represents the additional treatment effect (or difference in treatment effect) specifically for customers in Segment B (Facebook users).
The positive coefficient (3.154) indicates that the treatment effect of sending the coupon differs for customers in Segment B compared to those not in Segment B. In other words, Facebook users who receive the coupon tend to have an additional revenue increase compared to those who do not receive the coupon, controlling for other variables.

**Comparison between Segments:**

Comparing the coefficients for the interaction terms (test_coupon:segment_A and test_coupon:segment_B), we observe that the coefficient for Segment B (3.154) is larger than the coefficient for Segment A (2.756).
This suggests that the incremental revenue increase associated with sending the coupon is larger for Segment B (Facebook users) compared to Segment A (Instagram users).

In other words, Facebook users who receive the coupon exhibit a larger boost in revenue compared to Instagram users who receive the coupon.
In summary, while both Segment A and Segment B customers tend to have lower revenue compared to customers not in those segments, Facebook users (Segment B) show a larger incremental revenue increase when they receive the coupon compared to Instagram users (Segment A). This indicates that the coupon promotion may be more effective for acquiring revenue from Facebook users than from Instagram users.

# Question 6

Your team presents these results to the CEO, who commented that she wanted to see
how the proposed targeting strategies reach different demographic subgroups
compared to a mass-targeting campaign. You are now tasked to prepare a report that
shows the demographic reach of proposed targeting campaigns. For simplicity, I will
ask you to focus on Randy’s strategy for the duration of this question.

## Question 6(a)

Compare the differences in gender composition in Segment A compared to all
customers in the experiment. Include both point estimate, standard deviation, and
4
confidence interval. [Hint: for this question, use the combination of treatment and
control group when constructing the target segment and total customers.] (8
points)

In [ ]:
treat = df[df['segment_A'] == 1]['non_male'].mean()
control = df['non_male'].mean()

diff = treat - control

n_treat = len(df[df['segment_A'] == 1]['non_male'])
n_control = len(df['non_male'])

se = np.sqrt((np.std(df[df['segment_A'] == 1]['non_male'])**2 / n_treat) + (np.std(df['non_male'])**2 / n_control))

CI_lower = np.round((diff - 1.96 * se), 3)
CI_upper = np.round((diff + 1.96 * se), 3)

print('Difference between gender composition of Segment A, compared to all users: ', np.round(diff, 3))
print('Standard error: ', np.round(se, 3))
print('95% CI: [', CI_lower, ', ', CI_upper, '] \n')

Difference between gender composition of Segment A, compared to all users:  0.145
Standard error:  0.015
95% CI: [ 0.115 ,  0.175 ] 



## Question 6(b)

Compare the differences in income composition in Segment A compared to all
customers in the experiment. Include both point estimate, standard deviation, and
confidence interval. [Hint: you will need to first convert the variable into binary
variables.] (8 points)

In [ ]:
df['income'] = df['income'].replace({"high": 1, "low": 0})

In [ ]:
treat_income = df[df['segment_A'] == 1]['income'].mean()
control_income = df['income'].mean()

diff_income = treat_income - control_income

n_treat = len(df[df['segment_A'] == 1]['income'])
n_control = len(df['income'])

se_income = np.sqrt((np.std(df[df['segment_A'] == 1]['income'])**2 / n_treat) + (np.std(df['income'])**2 / n_control))

CI_lower_income = np.round((diff_income - 1.96 * se_income), 3)
CI_upper_income = np.round((diff_income + 1.96 * se_income), 3)

print('Difference between income composition of Segment A, compared to all users: ', np.round(diff_income, 3))
print('Standard error: ', np.round(se_income, 3))
print('95% CI: [', CI_lower_income, ', ', CI_upper_income, '] \n')

Difference between income composition of Segment A, compared to all users:  0.135
Standard error:  0.01
95% CI: [ 0.115 ,  0.154 ] 



## Question 6(c)

Conclude if there is a statistical significant difference in gender and income
composition in Segment A compared to all customers in the experiment. (4 points)

##Answer 6(c)

**Gender Composition:**

* The difference between the gender composition of Segment A compared to all users is 0.145.
* The standard error associated with this difference is 0.015.
* The 95% confidence interval for the difference in gender composition ranges from 0.115 to 0.175.

**Income Composition:**

* The difference between the income composition of Segment A compared to all users is 0.135.
* The standard error associated with this difference is 0.01.
* The 95% confidence interval for the difference in income composition ranges from 0.115 to 0.154.

**Conclusion:**

Both the difference in gender composition and income composition between Segment A and all users have confidence intervals that do not include zero.
This indicates that there is a statistically significant difference between Segment A and all users in terms of both gender and income composition.
Therefore, we can conclude that Segment A differs significantly from the overall customer population in terms of gender and income composition.
